In [1]:
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import random 
random.seed(13)

#visualization packages
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
%matplotlib inline
import seaborn as sns


In [ ]:
from pathlib import Path
d = Path().resolve().parent.parent.parent

d = str(d) + "/data/original/papers.csv"

df = pd.read_csv(d)
df.head()


## Cleaning and preprocessing 

In [3]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

papersArray = df.paper_text
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer() 

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalizedFromNouns = " ".join(lemma.lemmatize(word,'n') for word in punc_free.split())
    normalizedFromVerbs = " ".join(lemma.lemmatize(word,'v') for word in normalizedFromNouns.split())
    return normalizedFromVerbs


papersArrayClean = [clean(paper).split() for paper in papersArray]
#print(papersArrayClean)

# remove words that appear less then two times
from collections import defaultdict
frequency = defaultdict(int)

for text in papersArrayClean:
     for token in text:
         frequency[token] += 1

papersArrayClean = [[token for token in text if frequency[token] > 2]
          for text in papersArrayClean]

papersArrayCleanNoNumbers = []
# removing numbers 
for doc in papersArrayClean:
    numbers = list()
    for word in doc:
            if word.isnumeric(): numbers.append(word)
    numbers = list(numbers)
    for n in numbers:
        doc.remove(n)
    papersArrayCleanNoNumbers.append(doc)
        

papersArrayCleanNoLetter = []

# removing words with less than 4 characters 
for doc in papersArrayCleanNoNumbers:
    letters = list()
    for word in doc:
            if (len(word)<=3 ): letters.append(word)
    letters = list(letters)
    for n in letters:
        doc.remove(n)
    papersArrayCleanNoLetter.append(doc)


In [6]:
#creating a list of the papers paper_id, paper_text
papers =[]
for i in range(len(df)):
    paperString = " ".join(papersArrayCleanNoLetter[i])
    papers.append(paperString)

In [7]:
#set parameters 
n_features = 100 #features to use in the tifidf transformation 
n_topics = 8  #number of topics that will be displayed  ###RELEVANT PARAMETER 
n_top_words = 10 #number of words composing each topic 


In [8]:
#tfidf with n_features, removing stop words for English 
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=0.2,max_features=n_features,stop_words='english')
#fit tdidf with the paper_text
tfidf = tfidf_vectorizer.fit_transform(papers)

In [9]:
#transforming the tfidf representation into NMF matrix descomposition with n_topics as parameter
nmf = NMF(n_components=n_topics, random_state=0,alpha=.1, l1_ratio=.5).fit(tfidf)
##result in a 8*100 matrix


In [10]:
##Funtion to retrieve the n_top_words from the transformation 
##(taken from https://www.kaggle.com/dschniertshauer)
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [11]:
##Printing the topics 
print("Topics found via NMF:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

Topics found via NMF:

Topic #0:
algorithm function learn bind problem theorem matrix method sample loss

Topic #1:
network train unit input learn neural output weight error function

Topic #2:
model data distribution parameter sample variable estimate gaussian figure state

Topic #3:
image object feature label train learn representation classification figure model

Topic #4:
neuron input neural time network rate figure model noise information

Topic #5:
node graph algorithm variable structure network label problem random number

Topic #6:
kernel data feature train classification method space function vector matrix

Topic #7:
cluster data matrix algorithm point method number graph problem sample



In [12]:
#Finding relation between topics and papers
nmf_embedding = nmf.transform(tfidf)
nmf_embedding = (nmf_embedding - nmf_embedding.mean(axis=0))/nmf_embedding.std(axis=0)


## Getting the titles of the most related papers to the topics, so the decision of the topic name, could be less subjective

In [13]:
##Getting the titles of the most related papers to the topics, 
##so the decision of the topic name, could be less subjective
top_idx = np.argsort(nmf_embedding,axis=0)[-5:]
##8 topics
count = 0
for idxs in top_idx.T: 
    print("\nTopic {}:".format(count))
    for idx in idxs:
        print(df.iloc[idx]['title'])
    count += 1


Topic 0:
Stochastic Online AUC Maximization
Stochastic Gradient Methods for Distributionally Robust Optimization with f-divergences
Relax and Randomize : From Value to Algorithms
Fighting Bandits with a New Kind of Smoothness
Stochastic Variance Reduction Methods for Saddle-Point Problems

Topic 1:
Refining PID Controllers using Neural Networks
Dimensionality Reduction and Prior Knowledge in E-Set Recognition
Competition Among Networks Improves Committee Performance
A 'Neural' Network that Learns to Play Backgammon
Connectionist Learning of Expert Preferences by Comparison Training

Topic 2:
Modeling General and Specific Aspects of Documents with a Probabilistic Topic Model
Generalized Correspondence-LDA Models (GC-LDA) for Identifying Functional Regions in the Brain
Modeling User Rating Profiles For Collaborative Filtering
Comparing Bayesian models for multisensory cue combination without mandatory integration
A Bayesian Analysis of Dynamics in Free Recall

Topic 3:
Structure Driven 

# Assigning one topic to each paper

In [14]:
for i in range(len(df)):
    print(df.iloc[i].id, df.iloc[i].title,",",nmf_embedding.argmax(axis=1)[i])

1 Self-Organization of Associative Database and Its Applications , 3
10 A Mean Field Theory of Layer IV of Visual Cortex and Its Application to Artificial Neural Networks , 1
100 Storing Covariance by the Associative Long-Term Potentiation and Depression of Synaptic Strengths in the Hippocampus , 4
1000 Bayesian Query Construction for Neural Network Models , 1
1001 Neural Network Ensembles, Cross Validation, and Active Learning , 1
1002 Using a neural net to instantiate a deformable model , 3
1003 Plasticity-Mediated Competitive Learning , 5
1004 ICEG Morphology Classification using an Analogue VLSI Neural Network , 1
1005 Real-Time Control of a Tokamak Plasma Using Neural Networks , 1
1006 Pulsestream Synapses with Non-Volatile Analogue Amorphous-Silicon Memories , 1
1007 Learning to Play the Game of Chess , 1
1008 Multidimensional Scaling and Data Clustering , 7
1009 An experimental comparison of recurrent neural networks , 1
101 Training Multilayer Perceptrons with the Extended Kalm

1252 A Comparison between Neural Networks and other Statistical Techniques for Modeling the Relationship between Tobacco and Alcohol and Cancer , 1
1253 Improving the Accuracy and Speed of Support Vector Machines , 6
1254 The Effect of Correlated Input Data on the Dynamics of Learning , 1
1255 A Model of Recurrent Interactions in Primary Visual Cortex , 4
1256 The Learning Dynamcis of a Universal Approximator , 1
1257 Removing Noise in On-Line Search using Adaptive Batch Sizes , 1
1258 A Constructive RBF Network for Writer Adaptation , 1
1259 Are Hopfield Networks Faster than Conventional Computers? , 5
126 Computer Modeling of Associative Learning , 4
1260 Clustering via Concave Minimization , 7
1261 Continuous Sigmoidal Belief Networks Trained using Slice Sampling , 1
1262 Temporal Low-Order Statistics of Natural Sounds , 3
1263 Training Algorithms for Hidden Markov Models using Entropy Based Distance Functions , 0
1264 Hidden Markov Decision Trees , 5
1265 A Silicon Model of Amplitu

1500 Outcomes of the Equivalence of Adaptive Ridge with Least Absolute Shrinkage , 2
1501 Learning Curves for Gaussian Processes , 0
1502 Adding Constrained Discontinuities to Gaussian Process Models of Wind Fields , 2
1503 Learning from Dyadic Data , 7
1504 Example-Based Image Synthesis of Articulated Figures , 3
1505 Call-Based Fraud Detection in Mobile Communication Networks Using a Hierarchical Regime-Switching Model , 2
1506 Applications of Multi-Resolution Neural Networks to Mammography , 1
1507 Lazy Learning Meets the Recursive Least Squares Algorithm , 0
1508 Markov Processes on Curves for Automatic Speech Recognition , 2
1509 Experimental Results on Learning Stochastic Memoryless Policies for Partially Observable Markov Decision Processes , 0
151 Winner-Take-All Networks of O(N) Complexity , 4
1510 The Effect of Correlations on the Fisher Information of Population Codes , 4
1511 Learning Multi-Class Dynamics , 2
1512 Replicator Equations, Maximal Cliques, and Graph Isomorphism

1769 Understanding Stepwise Generalization of Support Vector Machines: a Toy Model , 6
177 Neural Network Star Pattern Recognition for Spacecraft Attitude Determination and Control , 4
1770 State Abstraction in MAXQ Hierarchical Reinforcement Learning , 0
1771 Leveraged Vector Machines , 6
1772 Monte Carlo POMDPs , 0
1773 Large Margin DAGs for Multiclass Classification , 5
1774 Effective Learning Requires Neuronal Remodeling of Hebbian Synapses , 1
1775 Reinforcement Learning for Spoken Dialogue Systems , 0
1776 LTD Facilitates Learning in a Noisy Environment , 1
1777 Acquisition in Autoshaping , 2
1778 Better Generative Models for Sequential Data Problems: Bidirectional Recurrent Mixture Density Networks , 2
1779 Data Visualization and Feature Selection: New Algorithms for Nongaussian Data , 1
178 A Model for Resolution Enhancement (Hyperacuity) in Sensory Representation , 2
1780 Learning Statistically Neutral Tasks without Expert Guidance , 1
1781 An Analog VLSI Model of Periodicity 

2018 The Fidelity of Local Ordinal Encoding , 3
2019 Learning Body Pose via Specialized Maps , 2
202 Learning in Higher-Order "Artificial Dendritic Trees , 5
2020 On Discriminative vs. Generative Classifiers: A comparison of logistic regression and naive Bayes , 2
2021 Group Redundancy Measures Reveal Redundancy Reduction in the Auditory Pathway , 4
2022 Learning Lateral Interactions for Feature Binding and Sensory Segmentation , 3
2023 Pranking with Ranking , 0
2024 Incorporating Invariances in Non-Linear Support Vector Machines , 6
2025 A Model of the Phonological Loop: Generalization and Binding , 1
2026 Modeling Temporal Structure in Classical Conditioning , 2
2027 TAP Gibbs Free Energy, Belief Propagation and Sparsity , 6
2028 Learning Discriminative Feature Transforms to Low Dimensions in Low Dimentions , 6
2029 Hyperbolic Self-Organizing Maps for Semantic Navigation , 5
203 Performance Comparisons Between Backpropagation Networks and Classification Trees on Three Real-World Appl

...

5751 Asynchronous Parallel Stochastic Gradient for Nonconvex Optimization , 0
5752 Distributed Submodular Cover: Succinctly Summarizing Massive Data , 0
5753 Probabilistic Line Searches for Stochastic Optimization , 0
5754 COEVOLVE: A Joint Point Process Model for Information Diffusion and Network Co-evolution , 5
5755 Linear Response Methods for Accurate Covariance Estimates from Mean Field Variational Bayes , 2
5756 Latent Bayesian melding for integrating individual and population models , 2
5757 Rapidly Mixing Gibbs Sampling for a Class of Factor Graphs Using Hierarchy Width , 5
5758 Automatic Variational Inference in Stan , 2
5759 Data Generation as Sequential Decision Making , 2
576 HARMONET: A Neural Net for Harmonizing Chorales in the Style of J. S. Bach , 1
5760 Stochastic Expectation Propagation , 0
5761 Deep learning with Elastic Averaging SGD , 0
5762 Competitive Distribution Estimation: Why is Good-Turing Good , 0
5763 Fast Convergence of Regularized Learning in Games , 0
5

5995 A Market Framework for Eliciting Private Data , 6
5996 Optimal Ridge Detection using Coverage Risk , 6
5997 Fast Distributed k-Center Clustering with Outliers on Massive Data , 7
5998 Orthogonal NMF through Subspace Exploration , 0
5999 Fast Classification Rates for High-dimensional Gaussian Generative Models , 2
6 A Neural-Network Solution to the Concentrator Assignment Problem , 4
60 The Hopfield Model with Multi-Level Neurons , 4
600 A Neural Network that Learns to Interpret Myocardial Planar Thallium Scintigrams , 1
6000 Predtron: A Family of Online Algorithms for General Prediction Problems , 0
6001 On the Optimality of Classifier Chain for Multi-label Classification , 0
6002 Smooth Interactive Submodular Set Cover , 0
6003 Tractable Bayesian Network Structure Learning with Bounded Vertex Cover Number , 5
6004 Secure Multi-party Differential Privacy , 0
6005 Adaptive Stochastic Optimization: From Sets to Paths , 0
6006 Learning structured densities via infinite dimensional ex

6258 Incremental Boosting Convolutional Neural Network for Facial Action Unit Recognition , 3
6259 Variational Bayes on Monte Carlo Steroids , 2
626 A Connectionist Symbol Manipulator That Discovers the Structure of Context-Free Languages , 1
6260 Learning User Perceived Clusters with Feature-Level Supervision , 4
6261 Visual Question Answering with Question Representation Update (QRU) , 3
6262 Adaptive Newton Method for Empirical Risk Minimization to Statistical Accuracy , 0
6263 Learning Deep Parsimonious Representations , 1
6264 Dialog-based Language Learning , 1
6265 A Sparse Interactive Model for Matrix Completion with Side Information , 0
6266 Ancestral Causal Inference , 5
6267 Regularized Nonlinear Acceleration , 0
6268 MetaGrad: Multiple Learning Rates in Online Learning , 0
6269 Graphical Time Warping for Joint Alignment of Multiple Curves , 5
627 History-Dependent Attractor Neural Networks , 4
6270 Stochastic Multiple Choice Learning for Training Diverse Deep Ensembles , 3
6

6526 Adaptive Maximization of Pointwise Submodular Functions With Budget Constraint , 0
6527 Conditional Image Generation with PixelCNN Decoders , 3
6528 Variational Autoencoder for Deep Learning of Images, Labels and Captions , 3
6529 Semiparametric Differential Graph Models , 5
653 Automatic Capacity Tuning of Very Large VC-Dimension Classifiers , 6
6530 Statistical Inference for Pairwise Graphical Models Using Score Matching , 2
6531 Testing for Differences in Gaussian Graphical Models:  Applications to Brain Connectivity , 2
6532 Tree-Structured Reinforcement Learning for Sequential Object Localization , 3
6533 A Non-generative Framework and Convex Relaxations for Unsupervised Learning , 0
6534 Scaling Factorial Hidden Markov Models: Stochastic Variational Inference without Messages , 1
6535 Nearly Isometric Embedding by Relaxation , 0
6536 Tracking the Best Expert in Non-stationary Stochastic Environments , 0
6537 A Probabilistic Model of Social Decision Making based on Reward Max

831 Learning Classification with Unlabeled Data , 3
832 Classifying Hand Gestures with a View-Based Distributed Representation , 3
833 Backpropagation without Multiplication , 1
834 Event-Driven Simulation of Networks of Spiking Neurons , 4
835 Statistics of Natural Images: Scaling in the Woods , 3
836 When will a Genetic Algorithm Outperform Hill Climbing , 0
837 Agnostic PAC-Learning of Functions on Analog Neural Nets , 5
838 Clustering with a Domain-Specific Distance Measure , 7
839 Discontinuous Generalization in Large Committee Machines , 1
84 Presynaptic Neural Information Processing , 5
840 Learning in Compositional Hierarchies: Inducing the Structure of Objects from Data , 5
841 Hoeffding Races: Accelerating Model Selection Search for Classification and Function Approximation , 2
842 A Network Mechanism for the Determination of Shape-From-Texture , 3
843 Robust Reinforcement Learning in Motion Planning , 0
844 Complexity Issues in Neural Computation and Learning , 1
845 Develop

In [20]:
text_file=open("NMF_paperID_topicID.csv","w")
text_file.write("%s, %s \n"%("paper_id","topic_id"))
for i in range(len(df)):
    text_file.write("%s, %s \n"%(df.iloc[i].id,nmf_embedding.argmax(axis=1)[i]))
    print(df.iloc[i].id,",",nmf_embedding.argmax(axis=1)[i])
text_file.close()    

1 , 3
10 , 1
100 , 4
1000 , 1
1001 , 1
1002 , 3
1003 , 5
1004 , 1
1005 , 1
1006 , 1
1007 , 1
1008 , 7
1009 , 1
101 , 5
1010 , 1
1011 , 1
1012 , 5
1013 , 4
1014 , 4
1015 , 1
1016 , 6
1017 , 2
1018 , 0
1019 , 2
102 , 2
1020 , 2
1021 , 1
1022 , 4
1023 , 4
1024 , 4
1025 , 1
1026 , 2
1027 , 2
1028 , 4
1029 , 1
103 , 5
1030 , 4
1031 , 1
1032 , 2
1033 , 1
1034 , 1
1035 , 2
1036 , 1
1037 , 1
1038 , 0
1039 , 1
104 , 3
1040 , 7
1041 , 6
1042 , 1
1043 , 1
1044 , 1
1045 , 3
1046 , 6
1047 , 3
1048 , 1
1049 , 4
105 , 4
1050 , 2
1051 , 1
1052 , 7
1053 , 4
1054 , 0
1055 , 1
1056 , 1
1057 , 4
1058 , 1
1059 , 1
106 , 1
1060 , 1
1061 , 0
1062 , 1
1063 , 2
1064 , 0
1065 , 2
1066 , 1
1067 , 1
1068 , 2
1069 , 4
107 , 3
1070 , 0
1071 , 4
1072 , 1
1073 , 1
1074 , 3
1075 , 1
1076 , 0
1077 , 1
1078 , 2
1079 , 3
108 , 2
1080 , 0
1081 , 1
1082 , 1
1083 , 1
1084 , 3
1085 , 4
1086 , 0
1087 , 1
1088 , 5
1089 , 1
109 , 1
1090 , 4
1091 , 0
1092 , 7
1094 , 1
1095 , 0
1096 , 5
1097 , 1
1098 , 2
1099 , 5
11 , 1
110 , 1
1

1886 , 3
1887 , 6
1888 , 1
1889 , 3
189 , 1
1890 , 5
1891 , 0
1892 , 2
1893 , 6
1894 , 4
1895 , 1
1896 , 7
1897 , 2
1898 , 3
1899 , 0
19 , 7
190 , 3
1900 , 2
1901 , 2
1902 , 4
1903 , 1
1904 , 2
1905 , 1
1906 , 0
1907 , 2
1908 , 2
1909 , 3
191 , 1
1910 , 4
1911 , 0
1912 , 3
1913 , 3
1914 , 4
1915 , 1
1916 , 3
1917 , 2
1918 , 1
1919 , 1
192 , 1
1920 , 1
1921 , 3
1922 , 6
1923 , 4
1924 , 0
1925 , 2
1926 , 2
1927 , 0
1928 , 6
1929 , 2
193 , 5
1930 , 6
1931 , 2
1932 , 5
1933 , 6
1934 , 1
1935 , 0
1936 , 0
1937 , 3
1938 , 2
1939 , 0
194 , 0
1940 , 5
1941 , 0
1942 , 1
1943 , 2
1944 , 0
1945 , 2
1946 , 6
1947 , 7
1948 , 0
1949 , 1
195 , 1
1950 , 3
1951 , 6
1952 , 7
1953 , 1
1954 , 7
1955 , 1
1956 , 2
1957 , 4
1958 , 6
1959 , 0
196 , 4
1960 , 1
1961 , 5
1962 , 3
1963 , 0
1964 , 6
1965 , 2
1966 , 2
1967 , 5
1968 , 1
1969 , 1
197 , 1
1970 , 0
1971 , 1
1972 , 4
1973 , 2
1974 , 2
1975 , 6
1976 , 6
1977 , 1
1978 , 0
1979 , 7
198 , 1
1980 , 7
1981 , 0
1982 , 4
1983 , 2
1984 , 0
1985 , 6
1986 , 0
1987

2919 , 2
292 , 5
2920 , 0
2921 , 3
2922 , 5
2923 , 2
2924 , 2
2925 , 0
2926 , 3
2927 , 3
2928 , 6
2929 , 2
293 , 1
2930 , 3
2931 , 4
2932 , 6
2933 , 0
2934 , 7
2935 , 2
2936 , 3
2937 , 3
2938 , 6
2939 , 3
294 , 1
2940 , 2
2941 , 2
2942 , 7
2943 , 0
2944 , 2
2945 , 6
2946 , 2
2947 , 6
2948 , 7
2949 , 3
295 , 1
2950 , 4
2951 , 0
2952 , 7
2953 , 5
2954 , 5
2955 , 1
2956 , 2
2957 , 3
2958 , 2
2959 , 0
296 , 1
2960 , 5
2961 , 6
2962 , 6
2963 , 0
2964 , 3
2965 , 2
2966 , 2
2967 , 1
2968 , 5
2969 , 7
297 , 2
2970 , 6
2971 , 0
2972 , 0
2973 , 5
2974 , 2
2975 , 2
2976 , 3
2977 , 7
2978 , 6
2979 , 0
298 , 1
2980 , 7
2981 , 3
2982 , 5
2983 , 0
2984 , 0
2985 , 7
2986 , 4
2987 , 0
2988 , 6
2989 , 5
299 , 3
2990 , 0
2991 , 6
2992 , 0
2993 , 0
2994 , 2
2995 , 0
2996 , 5
2997 , 5
2998 , 6
2999 , 6
3 , 5
30 , 1
300 , 5
3000 , 7
3001 , 0
3002 , 0
3003 , 1
3004 , 0
3005 , 6
3006 , 4
3007 , 0
3008 , 3
3009 , 3
301 , 1
3010 , 3
3011 , 4
3012 , 0
3013 , 0
3014 , 6
3015 , 7
3016 , 3
3017 , 2
3018 , 3
3019 , 

396 , 3
3960 , 0
3961 , 5
3962 , 0
3963 , 5
3964 , 0
3965 , 5
3966 , 5
3967 , 2
3968 , 3
3969 , 2
397 , 1
3970 , 4
3971 , 0
3972 , 0
3973 , 6
3974 , 0
3975 , 0
3976 , 2
3977 , 0
3978 , 5
3979 , 7
398 , 4
3980 , 1
3981 , 5
3982 , 3
3983 , 6
3984 , 0
3985 , 6
3986 , 0
3987 , 0
3988 , 6
3989 , 2
399 , 1
3990 , 6
3991 , 0
3992 , 2
3993 , 5
3994 , 0
3995 , 4
3996 , 6
3997 , 3
3998 , 0
3999 , 0
4 , 0
40 , 4
400 , 7
4000 , 6
4001 , 2
4002 , 2
4003 , 3
4004 , 0
4005 , 0
4006 , 0
4007 , 3
4008 , 3
4009 , 0
401 , 1
4010 , 5
4011 , 5
4012 , 3
4013 , 0
4014 , 0
4015 , 0
4016 , 3
4017 , 0
4018 , 0
4019 , 0
402 , 0
4020 , 4
4021 , 5
4022 , 7
4023 , 2
4024 , 0
4025 , 0
4026 , 0
4027 , 5
4028 , 0
4029 , 4
403 , 3
4030 , 2
4031 , 0
4032 , 0
4033 , 0
4034 , 6
4035 , 0
4036 , 5
4037 , 0
4038 , 7
4039 , 7
404 , 1
4040 , 6
4041 , 3
4042 , 6
4043 , 3
4044 , 5
4045 , 7
4046 , 5
4047 , 6
4048 , 0
4049 , 3
405 , 1
4050 , 2
4051 , 1
4052 , 5
4053 , 2
4054 , 0
4055 , 2
4056 , 4
4057 , 5
4058 , 5
4059 , 5
406 , 1

4825 , 6
4826 , 2
4827 , 3
4828 , 0
4829 , 2
483 , 1
4830 , 6
4831 , 3
4832 , 2
4833 , 7
4834 , 7
4835 , 0
4836 , 2
4837 , 2
4838 , 1
4839 , 6
484 , 1
4840 , 0
4841 , 6
4842 , 3
4843 , 0
4844 , 6
4845 , 7
4846 , 0
4847 , 6
4848 , 3
4849 , 2
485 , 1
4850 , 2
4851 , 5
4852 , 5
4853 , 7
4854 , 2
4855 , 0
4856 , 2
4857 , 5
4858 , 5
4859 , 0
486 , 2
4860 , 0
4861 , 2
4862 , 0
4863 , 0
4864 , 3
4865 , 7
4866 , 0
4867 , 6
4868 , 0
4869 , 2
487 , 2
4870 , 0
4871 , 4
4872 , 2
4873 , 4
4874 , 4
4875 , 4
4876 , 4
4877 , 2
4878 , 1
4879 , 2
488 , 2
4880 , 7
4881 , 3
4882 , 6
4883 , 0
4884 , 2
4885 , 2
4886 , 5
4887 , 3
4888 , 3
4889 , 2
489 , 0
4890 , 4
4891 , 0
4892 , 0
4893 , 6
4894 , 2
4895 , 6
4896 , 6
4897 , 6
4898 , 0
4899 , 5
49 , 1
490 , 1
4900 , 0
4901 , 5
4902 , 0
4903 , 0
4904 , 0
4905 , 0
4906 , 0
4907 , 2
4908 , 5
4909 , 0
491 , 4
4910 , 0
4911 , 0
4912 , 0
4913 , 2
4914 , 7
4915 , 7
4916 , 6
4917 , 0
4918 , 0
4919 , 0
492 , 2
4920 , 0
4921 , 2
4922 , 0
4923 , 7
4924 , 0
4925 , 5
4926

5723 , 0
5724 , 0
5725 , 0
5726 , 0
5727 , 0
5728 , 0
5729 , 0
573 , 1
5730 , 0
5731 , 0
5732 , 0
5733 , 0
5734 , 5
5735 , 0
5736 , 5
5737 , 0
5738 , 0
5739 , 0
574 , 3
5740 , 6
5741 , 7
5742 , 0
5743 , 0
5744 , 2
5745 , 0
5746 , 2
5747 , 5
5748 , 0
5749 , 0
575 , 1
5750 , 0
5751 , 0
5752 , 0
5753 , 0
5754 , 5
5755 , 2
5756 , 2
5757 , 5
5758 , 2
5759 , 2
576 , 1
5760 , 0
5761 , 0
5762 , 0
5763 , 0
5764 , 1
5765 , 6
5766 , 0
5767 , 4
5768 , 5
5769 , 2
577 , 1
5770 , 2
5771 , 2
5772 , 6
5773 , 3
5774 , 3
5775 , 1
5776 , 3
5777 , 3
5778 , 3
5779 , 3
578 , 4
5780 , 3
5781 , 3
5782 , 1
5783 , 1
5784 , 1
5785 , 3
5786 , 2
5787 , 1
5788 , 1
5789 , 2
579 , 1
5790 , 4
5791 , 5
5792 , 2
5793 , 2
5794 , 2
5795 , 0
5796 , 2
5797 , 1
5798 , 6
5799 , 7
58 , 1
580 , 1
5800 , 7
5801 , 0
5802 , 7
5803 , 5
5804 , 0
5805 , 0
5806 , 2
5807 , 0
5808 , 5
5809 , 5
581 , 4
5810 , 6
5811 , 0
5812 , 0
5813 , 6
5814 , 7
5815 , 5
5816 , 0
5817 , 0
5818 , 2
5819 , 0
582 , 3
5820 , 4
5821 , 0
5822 , 0
5823 , 0
5824

6602 , 6
6603 , 2
661 , 2
662 , 2
663 , 0
664 , 1
665 , 1
666 , 1
667 , 1
668 , 1
669 , 2
67 , 1
670 , 3
671 , 4
672 , 3
673 , 1
674 , 1
675 , 4
676 , 1
677 , 4
678 , 1
679 , 1
68 , 1
680 , 4
681 , 1
682 , 1
683 , 1
684 , 4
685 , 7
686 , 0
687 , 1
689 , 4
69 , 5
690 , 0
691 , 1
692 , 1
693 , 4
694 , 4
695 , 1
696 , 1
697 , 2
699 , 1
7 , 3
70 , 1
700 , 2
701 , 1
702 , 1
703 , 1
704 , 1
705 , 4
706 , 1
707 , 2
708 , 1
709 , 3
71 , 2
710 , 2
712 , 0
713 , 1
714 , 1
715 , 7
716 , 0
717 , 4
718 , 1
719 , 7
72 , 1
720 , 1
721 , 0
722 , 1
723 , 5
724 , 1
725 , 4
726 , 1
727 , 3
728 , 4
729 , 4
73 , 4
730 , 4
731 , 1
733 , 2
734 , 1
735 , 7
736 , 7
737 , 0
738 , 3
739 , 4
74 , 5
740 , 3
741 , 1
742 , 0
743 , 4
744 , 1
745 , 7
746 , 1
747 , 2
748 , 5
749 , 1
75 , 1
750 , 1
751 , 2
752 , 3
753 , 1
754 , 1
755 , 4
756 , 4
757 , 1
758 , 1
759 , 1
76 , 5
760 , 4
761 , 2
762 , 1
763 , 1
764 , 0
765 , 1
766 , 4
767 , 2
768 , 3
769 , 1
77 , 4
770 , 1
771 , 3
772 , 0
773 , 0
774 , 1
775 , 4
776 , 4
777